In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset_path = '/kaggle/input/chest-x-ray-dataset-4-categories/Chest X_Ray Dataset/'

In [ ]:
image_paths = []
categories = []

categories_list = ['COVID19', 'NORMAL', 'PNEUMONIA', 'TURBERCULOSIS']  

for category in categories_list:
    category_path = os.path.join(dataset_path, category)
    
    if os.path.exists(category_path):
        
        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name)
            image_paths.append(image_path)
            categories.append(category)

df = pd.DataFrame({'image_path': image_paths, 'category': categories})

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['category'].unique()

In [ ]:
df['category'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

category_counts = df['category'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=category_counts.index, y=category_counts.values, palette="viridis")
plt.title("Count of Images per Category")
plt.xlabel("Category")
plt.ylabel("Number of Images")
plt.show()

plt.figure(figsize=(8, 6))
plt.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("viridis", len(category_counts)))
plt.title("Proportion of Images per Category")
plt.show()

In [ ]:
import random
from PIL import Image

num_images_per_category = 5

plt.figure(figsize=(15, 10))

for i, category in enumerate(df['category'].unique()):

    category_images = df[df['category'] == category]['image_path']
    
    selected_images = random.sample(list(category_images), num_images_per_category)
    
    for j, image_path in enumerate(selected_images):

        img = Image.open(image_path)
        
        plt.subplot(len(df['category'].unique()), num_images_per_category, i * num_images_per_category + j + 1)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.title(category if j == 0 else "") 

plt.tight_layout()
plt.show()

In [ ]:
def encoder_block(inputs, num_filters): 

	x = tf.keras.layers.Conv2D(num_filters, 
							3, 
							padding = 'valid')(inputs) 
	x = tf.keras.layers.Activation('relu')(x) 
	
	x = tf.keras.layers.Conv2D(num_filters, 
							3, 
							padding = 'valid')(x) 
	x = tf.keras.layers.Activation('relu')(x) 

	x = tf.keras.layers.MaxPool2D(pool_size = (2, 2), 
								strides = 2)(x) 
	
	return x

In [ ]:
from tensorflow.keras.layers import Lambda

def decoder_block(inputs, skip_features, num_filters): 
   
    x = tf.keras.layers.Conv2DTranspose(num_filters, 
                                        (2, 2), 
                                        strides = 2, 
                                        padding = 'valid')(inputs) 
    
   
    x = Lambda(lambda x: tf.image.resize(x, size=(skip_features.shape[1], skip_features.shape[2])))(x)

    x = tf.keras.layers.Concatenate()([x, skip_features]) 
    
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 

    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
    
    return x

In [ ]:
import tensorflow as tf 

def unet_model(input_shape = (256, 256, 3), num_classes = 2): 
	inputs = tf.keras.layers.Input(input_shape) 
	
	s1 = encoder_block(inputs, 64) 
	s2 = encoder_block(s1, 128) 
	s3 = encoder_block(s2, 256) 
	s4 = encoder_block(s3, 512) 
	
	b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(s4) 
	b1 = tf.keras.layers.Activation('relu')(b1) 
	b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(b1) 
	b1 = tf.keras.layers.Activation('relu')(b1) 
	
	s5 = decoder_block(b1, s4, 512) 
	s6 = decoder_block(s5, s3, 256) 
	s7 = decoder_block(s6, s2, 128) 
	s8 = decoder_block(s7, s1, 64) 
	
	outputs = tf.keras.layers.Conv2D(num_classes, 
									1, 
									padding = 'valid', 
									activation = 'sigmoid')(s8) 
	
	model = tf.keras.models.Model(inputs = inputs, 
								outputs = outputs, 
								name = 'U-Net') 
	return model 

if __name__ == '__main__': 
	model = unet_model(input_shape=(572, 572, 3), num_classes=2) 
	model.summary()

In [ ]:
import numpy as np 
from PIL import Image 
from tensorflow.keras.preprocessing import image 
 
img = Image.open('/kaggle/input/chest-x-ray-dataset-4-categories/Chest X_Ray Dataset/COVID19/COVID19(0).jpg')

img = img.convert('RGB')

img = img.resize((572, 572)) 
img_array = image.img_to_array(img) 

img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0

model = unet_model(input_shape=(572, 572, 3), num_classes=4)

predictions = model.predict(img_array)

predictions = np.squeeze(predictions, axis=0)
predictions = np.argmax(predictions, axis=-1)

predictions = Image.fromarray(np.uint8(predictions * 255))
predictions = predictions.resize((img.width, img.height))

predictions.save('predicted_image.jpg') 
predictions

In [ ]:
img = Image.open('/kaggle/input/chest-x-ray-dataset-4-categories/Chest X_Ray Dataset/NORMAL/IM-0001-0001.jpeg')

img = img.convert('RGB')

img = img.resize((572, 572)) 
img_array = image.img_to_array(img) 

img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0

model = unet_model(input_shape=(572, 572, 3), num_classes=2)

predictions = model.predict(img_array)

predictions = np.squeeze(predictions, axis=0)
predictions = np.argmax(predictions, axis=-1)

predictions = Image.fromarray(np.uint8(predictions * 255))
predictions = predictions.resize((img.width, img.height))

predictions.save('predicted_image.jpg') 
predictions

In [ ]:
img = Image.open('/kaggle/input/chest-x-ray-dataset-4-categories/Chest X_Ray Dataset/PNEUMONIA/person1000_bacteria_2931.jpeg')

img = img.convert('RGB')

img = img.resize((572, 572)) 
img_array = image.img_to_array(img) 

img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0

model = unet_model(input_shape=(572, 572, 3), num_classes=2)

predictions = model.predict(img_array)

predictions = np.squeeze(predictions, axis=0)
predictions = np.argmax(predictions, axis=-1)

predictions = Image.fromarray(np.uint8(predictions * 255))
predictions = predictions.resize((img.width, img.height))

predictions.save('predicted_image.jpg') 
predictions

In [ ]:
img = Image.open('/kaggle/input/chest-x-ray-dataset-4-categories/Chest X_Ray Dataset/TURBERCULOSIS/Tuberculosis-1.png')

img = img.convert('RGB')

img = img.resize((572, 572)) 
img_array = image.img_to_array(img) 

img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0

model = unet_model(input_shape=(572, 572, 3), num_classes=2)

predictions = model.predict(img_array)

predictions = np.squeeze(predictions, axis=0)
predictions = np.argmax(predictions, axis=-1)

predictions = Image.fromarray(np.uint8(predictions * 255))
predictions = predictions.resize((img.width, img.height))

predictions.save('predicted_image.jpg') 
predictions

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['category_encoded'] = label_encoder.fit_transform(df['category'])

In [ ]:
df

In [ ]:
df = df[['image_path', 'category_encoded']]

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(df[['image_path']], df['category_encoded'])

In [ ]:
df_resampled = pd.DataFrame(X_resampled, columns=['image_path'])
df_resampled['category_encoded'] = y_resampled

In [ ]:
print("\nClass distribution after oversampling:")
print(df_resampled['category_encoded'].value_counts())

In [ ]:
df_resampled

In [ ]:
df_resampled['category_encoded'] = df_resampled['category_encoded'].astype(str)

import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Define the U-Net model (assuming encoder_block and decoder_block are defined)
def unet_model(input_shape=(256, 256, 3), num_classes=2):
    inputs = tf.keras.layers.Input(input_shape) 
    
    s1 = encoder_block(inputs, 64) 
    s2 = encoder_block(s1, 128) 
    s3 = encoder_block(s2, 256) 
    s4 = encoder_block(s3, 512) 
    
    b1 = tf.keras.layers.Conv2D(1024, 3, padding='valid')(s4) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding='valid')(b1) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
    
    s5 = decoder_block(b1, s4, 512) 
    s6 = decoder_block(s5, s3, 256) 
    s7 = decoder_block(s6, s2, 128) 
    s8 = decoder_block(s7, s1, 64) 
    
    outputs = tf.keras.layers.Conv2D(num_classes, 1, padding='valid', activation='sigmoid')(s8) 
    
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='U-Net') 
    return model 

# Function to create folders for each class
def create_folders():
    class_names = ['COVID19', 'NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']
    for class_name in class_names:
        class_dir = os.path.join('/kaggle/working/', class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

# Function to generate segmentation masks
def generate_segmentation_masks(df, model, img_size=(256, 256)):
    # Define the class names based on your categories
    class_names = ['COVID19', 'NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']
    
    # Ensure category_encoded is of integer type
    df['category_encoded'] = pd.to_numeric(df['category_encoded'], errors='coerce')
    
    for idx, row in df.iterrows():
        image_path = row['image_path']
        class_label = row['category_encoded']  # This should now be an integer
        
        # Ensure the class label is within the correct range
        if class_label < 0 or class_label >= len(class_names):
            print(f"Warning: Invalid class label {class_label} for image {image_path}. Skipping.")
            continue
        
        # Define the class name based on the label (integer value)
        class_name = class_names[class_label]  # Map class_label to class name
        
        # Load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        img_array = tf.expand_dims(img_array, axis=0)  # Add batch dimension
        
        # Predict segmentation mask using the trained model
        pred_mask = model.predict(img_array)
        pred_mask = tf.squeeze(pred_mask, axis=0)  # Remove batch dimension
        
        # If the model has 2 channels (e.g., for binary classification), select one channel or threshold
        if pred_mask.shape[-1] == 2:
            # Choose the channel of interest (e.g., channel for the 'COVID19' class)
            pred_mask = np.argmax(pred_mask, axis=-1)  # Convert to single-channel mask (class prediction)
        
        # If it's a binary mask, ensure it's binary (0 or 1)
        pred_mask = (pred_mask > 0.5).astype(np.uint8)
        
        # Add the channel dimension to make it (height, width, 1)
        pred_mask = np.expand_dims(pred_mask, axis=-1)
        
        # Create the directory for the class if it doesn't exist
        output_dir = f'/kaggle/working/{class_name}'
        os.makedirs(output_dir, exist_ok=True)
        
        # Save the predicted mask as an image
        mask_image_path = os.path.join(output_dir, f'{os.path.basename(image_path)}_mask.png')
        tf.keras.preprocessing.image.save_img(mask_image_path, pred_mask)
        
        print(f"Saved mask for {image_path} to {mask_image_path}")

# Now, you can use the model and df_resampled to generate masks
# Example usage:
# Assuming df_resampled is your dataframe
model = unet_model(input_shape=(256, 256, 3), num_classes=2)  # Adjust num_classes for your use case
generate_segmentation_masks(df_resampled, model)


In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image

def unet_model(input_shape=(256, 256, 3), num_classes=2):
    inputs = tf.keras.layers.Input(input_shape) 
    
    s1 = encoder_block(inputs, 64) 
    s2 = encoder_block(s1, 128) 
    s3 = encoder_block(s2, 256) 
    s4 = encoder_block(s3, 512) 
    
    b1 = tf.keras.layers.Conv2D(1024, 3, padding='valid')(s4) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding='valid')(b1) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
    
    s5 = decoder_block(b1, s4, 512) 
    s6 = decoder_block(s5, s3, 256) 
    s7 = decoder_block(s6, s2, 128) 
    s8 = decoder_block(s7, s1, 64) 
    
    outputs = tf.keras.layers.Conv2D(num_classes, 1, padding='valid', activation='sigmoid')(s8) 
    
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='U-Net') 
    return model

def create_folders():
    class_names = ['COVID19', 'NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']
    for class_name in class_names:
        class_dir = os.path.join('/kaggle/working/', class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

def generate_segmentation_masks(df, model, img_size=(256, 256), num_images_per_class=100):
  
    class_names = ['COVID19', 'NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']
    
    df['category_encoded'] = pd.to_numeric(df['category_encoded'], errors='coerce')
    
    for class_label in range(len(class_names)):
        class_df = df[df['category_encoded'] == class_label].sample(n=num_images_per_class, random_state=42)
        
        for idx, row in class_df.iterrows():
            image_path = row['image_path']
            class_label = row['category_encoded']  
          
            class_name = class_names[class_label]  
            
            img = tf.keras.preprocessing.image.load_img(image_path, target_size=img_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            img_array = tf.expand_dims(img_array, axis=0)  
            
            pred_mask = model.predict(img_array)
            pred_mask = tf.squeeze(pred_mask, axis=0) 
            
            if pred_mask.shape[-1] == 2:
                
                pred_mask = np.argmax(pred_mask, axis=-1)  
            
            pred_mask = (pred_mask > 0.5).astype(np.uint8)
            
            pred_mask = np.expand_dims(pred_mask, axis=-1)
            
            output_dir = f'/kaggle/working/{class_name}'
            os.makedirs(output_dir, exist_ok=True)
            
            mask_image_path = os.path.join(output_dir, f'{os.path.basename(image_path)}_mask.png')
            tf.keras.preprocessing.image.save_img(mask_image_path, pred_mask)
            
            print(f"Saved mask for {image_path} to {mask_image_path}")


model = unet_model(input_shape=(256, 256, 3), num_classes=2) 
generate_segmentation_masks(df_resampled, model)

In [ ]:
df_resampled

In [ ]:
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

import warnings
warnings.filterwarnings("ignore")

print ('check')

In [ ]:
df_resampled['category_encoded'] = df_resampled['category_encoded'].astype(str)

In [ ]:
train_df_new, temp_df_new = train_test_split(
    df_resampled,
    train_size=0.8,  
    shuffle=True,
    random_state=42,
    stratify=df_resampled['category_encoded']  
)

valid_df_new, test_df_new = train_test_split(
    temp_df_new,
    test_size=0.5,  
    shuffle=True,
    random_state=42,
    stratify=temp_df_new['category_encoded'] 
)

In [ ]:
batch_size = 16
img_size = (256, 256)
channels = 3  
img_shape = (img_size[0], img_size[1], channels)

tr_gen = ImageDataGenerator(rescale=1./255)  
ts_gen = ImageDataGenerator(rescale=1./255)

train_gen_new = tr_gen.flow_from_dataframe(
    train_df_new,
    x_col='image_path',  
    y_col='category_encoded',     
    target_size=img_size,
    class_mode='sparse',  
    color_mode='rgb', 
    shuffle=True,
    batch_size=batch_size
)

valid_gen_new = ts_gen.flow_from_dataframe(
    valid_df_new,
    x_col='image_path',  
    y_col='category_encoded',     
    target_size=img_size,
    class_mode='sparse',  
    color_mode='rgb', 
    shuffle=True,
    batch_size=batch_size
)

test_gen_new = ts_gen.flow_from_dataframe(
    test_df_new,
    x_col='image_path', 
    y_col='category_encoded',    
    target_size=img_size,
    class_mode='sparse',  
    color_mode='rgb', 
    shuffle=False,  
    batch_size=batch_size
)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("Using GPU")
else:
    print("Using CPU")

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
def unet_classification_model(input_size=(256, 256, 3), num_classes=4):
    inputs = layers.Input(input_size)

    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D((2, 2))(conv3)

    bottleneck = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    bottleneck = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(bottleneck)

    upconv3 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(bottleneck)
    concat3 = layers.concatenate([upconv3, conv3])
    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(concat3)
    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)

    upconv2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4)
    concat2 = layers.concatenate([upconv2, conv2])
    conv5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(concat2)
    conv5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    upconv1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv5)
    concat1 = layers.concatenate([upconv1, conv1])
    conv6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat1)
    conv6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    gap = layers.GlobalAveragePooling2D()(conv6)  

    output = layers.Dense(num_classes, activation='softmax')(gap)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

model = unet_classification_model(input_size=(256, 256, 3), num_classes=4)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_gen_new,  
    epochs=5,
    batch_size=16,
    validation_data=valid_gen_new,  
    steps_per_epoch=train_gen_new.samples // train_gen_new.batch_size,
    validation_steps=valid_gen_new.samples // valid_gen_new.batch_size,
    verbose=1
)

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.show()

In [ ]:
y_pred_probs = model.predict(test_gen_new)
y_pred = np.argmax(y_pred_probs, axis=1) 

In [ ]:
y_true = test_gen_new.classes

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_gen_new.class_indices.keys(), yticklabels=test_gen_new.class_indices.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=test_gen_new.class_indices.keys())
print("Classification Report:\n", report)

In [ ]:
def attention_gate(x, g, inter_channels):
    """Attention Gate Implementation."""
    theta_x = layers.Conv2D(inter_channels, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = layers.Conv2D(inter_channels, (1, 1), strides=(1, 1), padding='same')(g)
    add_xg = layers.Add()([theta_x, phi_g])
    relu_xg = layers.Activation('relu')(add_xg)
    psi = layers.Conv2D(1, (1, 1), strides=(1, 1), padding='same')(relu_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    attention = layers.Multiply()([x, sigmoid_xg])
    return attention

def unet_with_attention(input_size=(256, 256, 3), num_classes=4):
    inputs = layers.Input(input_size)

    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D((2, 2))(conv3)

    bottleneck = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    bottleneck = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(bottleneck)

    upconv3 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(bottleneck)
    attn3 = attention_gate(conv3, upconv3, inter_channels=128)
    concat3 = layers.concatenate([upconv3, attn3])
    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(concat3)
    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)

    upconv2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4)
    attn2 = attention_gate(conv2, upconv2, inter_channels=64)
    concat2 = layers.concatenate([upconv2, attn2])
    conv5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(concat2)
    conv5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    upconv1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv5)
    attn1 = attention_gate(conv1, upconv1, inter_channels=32)
    concat1 = layers.concatenate([upconv1, attn1])
    conv6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat1)
    conv6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    gap = layers.GlobalAveragePooling2D()(conv6)  
    output = layers.Dense(num_classes, activation='softmax')(gap)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

model = unet_with_attention(input_size=(256, 256, 3), num_classes=4)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_gen_new,  
    epochs=5,
    batch_size=16,
    validation_data=valid_gen_new,  
    steps_per_epoch=train_gen_new.samples // train_gen_new.batch_size,
    validation_steps=valid_gen_new.samples // valid_gen_new.batch_size,
    verbose=1
)

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.show()

In [ ]:
y_pred_probs = model.predict(test_gen_new)
y_pred = np.argmax(y_pred_probs, axis=1) 

In [ ]:
y_true = test_gen_new.classes

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_gen_new.class_indices.keys(), yticklabels=test_gen_new.class_indices.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
report = classification_report(y_true, y_pred, target_names=test_gen_new.class_indices.keys())
print("Classification Report:\n", report)